In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Random forest là gì ?

## Định nghĩa.

Random forest là một phương pháp thống kê mô hình hóa bằng máy (machine learning statistic) dùng để phục vụ các mục đích phân loại, hồi quy và các nhiệm vụ khác bằng cách xây dựng nhiều cây quyết định (Decision tree).

[_Nguồn_](https://jgac.vn/journal/article/view/344#:~:text=Random%20forest%20l%C3%A0%20m%E1%BB%99t%20ph%C6%B0%C6%A1ng,quy%E1%BA%BFt%20%C4%91%E1%BB%8Bnh%20(Decision%20tree).)

Random Forest được ghi nhận hiệu quả hơn so với thuật toán phân loại khác thường được sử dụng vì có khả năng tìm ra thuộc tính nào quan trọng hơn so với những thuộc tính khác.

</br>

![](https://res.cloudinary.com/dyd911kmh/image/upload/v1677239993/image5_c214968fd6.png)

<br>

**Ưu điểm:**

Random forests được coi là một phương pháp chính xác và mạnh mẽ vì số cây quyết định tham gia vào quá trình này. Thuật toán có thể được sử dụng trong cả hai vấn đề phân loại và hồi quy.

<br>

**Nhược điểm:**

Random forests chậm tạo dự đoán bởi vì nó có nhiều cây quyết định. Bất cứ khi nào nó đưa ra dự đoán, tất cả các cây trong rừng phải đưa ra dự đoán cho cùng một đầu vào cho trước và sau đó thực hiện bỏ phiếu trên đó. Toàn bộ quá trình này tốn rất nhiều thời gian.

Mô hình khó hiểu hơn so với cây quyết định, nơi ta có thể dễ dàng đưa ra quyết định bằng cách đi theo đường dẫn trong cây.

## Cách thức hoạt động.

1. Chọn các mẫu ngẫu nhiên từ tập dữ liệu đã cho.

2. Thiết lập cây quyết định cho từng mẫu và nhận kết quả dự đoán từ mỗi quyết định cây.

3. Bỏ phiếu cho mỗi kết quả dự đoán.

4. Chọn kết quả được dự đoán nhiều nhất làm dự đoán cuối cùng.

<br>

![](https://images.viblo.asia/33adb558-67ba-4262-809e-3f8a8348e0c8.png)

## Ứng dụng thực tế.

Chủ yếu áp dụng vào việc giải các bài toán liên quan đến phân loại và hồi quy.

<br>

**Ví dụ:** Phân loại bệnh hoặc dự đoán giá cả hàng hóa trong tương lai.

# Hướng phát triển của nhóm.

Áp dụng Numba, CUDA để chạy song song cho thuật toán Random forest nhằm mục đích cài thiện tốc độ trong quá trình sử dụng. Tiết kiệm thời gian huấn luyện và tận dụng tốt tài nguyên máy nhất có thể.

# Nguồn tài liệu.

Nhóm tìm được thuật toán Random Forest từ một [nguồn trên github](https://github.com/harrypnh/random-forest-from-scratch), họ implement thuật toán với 2 mục đích chính: chẩn đoán bệnh ung thư vú và đánh giá ô tô ( Car Evalution ).

<br>

**Tại sao lại chọn từ nguồn này?**

Vì họ xây dựng thuật toán Random Forest theo hàm chứ không xây theo class, điều này khiến cho quá trình làm việc với numba trở nên dễ dàng hơn.


# Dữ liệu đầu vào.

Về phần dataset, có lẽ nhóm sẽ tìm một dataset nào đó đủ lớn và dễ hiểu để training model trước. Việc này sẽ dễ hơn là dùng lại dataset có sẵn vì các kiến thức về xe ô tô và y tế nhóm chưa có nhiều kinh nghiệm nên việc diễn giải sẽ khó hơn.

# Mục tiêu cần đạt được.

xây dựng ra tất cả 4 version:
- V1 - là thuật toán Random Forest chạy tuần tự.
- V2 - là thuật toán Random Forest chạy song song bằng CPU.
- V3 - là thuật toán Random Forest chạy song song bằng GPU.
- V4 - là thuật toán Random Forest chạy song song sử dụng shared memory.

<br>

Ở mức kỳ vọng 100% - nhóm mong muốn có thể xây dựng được thuật toán đến V3 ( GPU ) và tốc độ có sự cải thiện khi chạy với dataset đủ lớn.

<br>

Ở mức kỳ vọng 125% - nhóm mong muốn xây dựng được thuật toán đến V4 và có thể ứng dụng thuật toán của nhóm vào để thử giải quyết một bài toán thực tế nào đó do các thành viên nhóm đề cử ( ví dụ chuẩn đoán bệnh ).


# Utility functions

In [ ]:
# Hàm hỗ trợ phân tách tập train - test

def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

## Khái quát

Để xây dựng thuật toán Random forest ( tuần tự ) thì ở đây, nhóm cần build 2 phần từ là:

1. **DecisionTree:** Xử lý các vấn đề liên quan đến cây quyết định.

2. **RandomForest:** Xử lý các vấn đề liên quan đến random forest.



# 1. Random Forest tuần tự **[ V1 ]**

## Decision Tree

In [ ]:
import numpy
import pandas
import random


'''
Hàm này kiểm tra xem có tiếp tục còn nhánh để rẽ nữa hay k,
nếu k thì trả về True, ngược lại trả về False.
'''

def checkPurity(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False


'''
Hàm này kiểm tra tất cả các nhãn cuối cùng của các cây quyết định,
sau đó chọn ra nhãn xuất hiện nhiều nhất và gắn cho đối tượng cần
dự đoán.
'''

def classifyData(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]


'''
Hàm này mực đích tìm ra các ứng cử viên sáng giá cho việc rẽ nhánh. Nhận vào
data và randomAttributes.

Hàm sẽ kiểm tra như comment:
'''

def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape

    # lấy ra số lượng cột có trong data.
    columnsIndices = list(range(columns - 1))

    # kiểm tra nếu randomAttributes != None & số lượng randomAttributes <=
    # số cột tối đa của data truyền vào thì gán
    # columnsIndices = randomAttributes
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes

    # với mỗi cột trong columnsIndices, ta lấy ra tất cả dòng giá trị của cột đó
    # và lọc ra các giá trị độc nhất.
    # nếu chỉ có duy nhất 1 giá trị độc nhất thì ra gán
    # potentialSplits[column] = uniqueValues tức ứng cử viên cho cột ta vừa chọn
    # là giá trị đó.
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in range(len(uniqueValues)):
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits

'''

'''

def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]


# //
def calculateEntropy(data):
    _, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()
    return sum(probabilities * -numpy.log2(probabilities))



def calculateOverallEntropy(dataBelow, dataAbove):
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)



def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in range(randomSplits): # //
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue


'''
Hàm này dùng để xây dựng cây quyết định. Nhận vào các tham số: data,
độ sâu hiện tại, số lượng mẫu tối thiểu để tiếp tục phân tách,
độ sâu tối đa, số lượng mẫu tối thiểu để tiếp tục split,
số lần phân tách ngẫu nhiên.
'''

def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):


    # Nếu độ sâu hiện tại = 0, ta kiểm tra xem:

    # Nếu randomAttributes != None & randomAttributes <= số lượng các cột - cột label thì
    # ta gán randomAttributes = các cột bốc một cách ngẫu nhiên bằng hàm random.

    # Ngược lại, randomAttributes = None.

    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None


    # Với độ sâu != 0, ta kiểm tra xem nếu hội tự các yếu tố: Purity = True hoặc
    # số lượng mẫu ít hơn số lượng mẫu tối thiểu hoặc
    # độ sâu đã đạt tối đa
    # ==> ta cho mô hình phân loại luôn.

    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)

    # Ngược lại, ta đi sâu hơn currentDepth += 1, lấy dánh sách các ứng cử viên
    # sáng giá để rẽ nhánh.
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree



def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)



def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions



def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Random forest

In [ ]:

# Lấy mẫu Bootstrap là một kỹ thuật cơ bản được sử dụng trong việc xây dựng các khu rừng ngẫu nhiên.

# Kỹ thuật này sẽ giúp ta tạo ra các decision tree một cách đa dạng hơn.

def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = numpy.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]


# randomAttributes: số lượng thuộc tính ngẫu nhiên đc chọn cho mỗi lần phân tách.
#                   việc này giúp giảm overfiting & tăng tính đa dạng giữa các cây
#                   trong rừng.

# randomSplits:     chứa các splitting point cho mỗi randomAttributes --> đa dạng hóa,
#                   hạn chế overfitting.
def createRandomForest(dataFrame, bootstrapSize, randomAttributes, randomSplits, numTree = 20, maxDepth = 1000):
    forest = []
    for i in range(numTree):
        bootstrappedDF = bootstrapSample(dataFrame, bootstrapSize)
        decisionTree = buildDecisionTree(bootstrappedDF, maxDepth = maxDepth, randomAttributes = randomAttributes, randomSplits = randomSplits)
        forest.append(decisionTree)
    return forest



# Gom lại hết các kết quả dự đoán từ các cây quyết định và lấy mode để  cho ra kq cuối cùng.
def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    for i in range(len(randomForest)):
        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pandas.DataFrame(predictions)
    return predictions.mode(axis = 1)[0]

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Sample run

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/iris.csv")

# label encoder
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
df['species']= label_encoder.fit_transform(df['species'])

%time
randomForest = createRandomForest(df, bootstrapSize = 10, randomAttributes = 10, randomSplits = 50, numTree = 30, maxDepth = 3)
randomForestTestResults = randomForestPredictions(df, randomForest)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [ ]:
accuracy = calculateAccuracy(randomForestTestResults, df.iloc[:, -1]) * 100

In [ ]:
accuracy

96.0

# 2. Song song hóa sử dụng CPU **[ V2 ]**

## Decision Tree V2.0

In [ ]:
import numpy
import pandas
import random
from numba import njit, prange



def checkPurity(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False



def classifyData(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]



def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in prange(len(uniqueValues)): # // ở đây bằng prange
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits



def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]


njit(parallel=True)
def calculateEntropy(data):
    _, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()

    # // ở đây, thay thế hàm sum có sẵn trong python bằng vòng lặp sử dụng prange
    entropy = 0.0
    for i in prange(len(probabilities)):
        entropy -= probabilities[i] * np.log2(probabilities[i])
    return entropy



def calculateOverallEntropy(dataBelow, dataAbove):
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)



def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in prange(randomSplits): # // vòng for này bằng prange
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue



def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree



def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)



def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions



def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Random Forest V2.0

In [ ]:
def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = numpy.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]



def createRandomForest(dataFrame, bootstrapSize, randomAttributes, randomSplits, forestSize = 20, treeMaxDepth = 1000):
    forest = []
    for i in prange(forestSize): # // hóa với prange thay range
        bootstrappedDataFrame = bootstrapSample(dataFrame, bootstrapSize)
        decisionTree = buildDecisionTree(bootstrappedDataFrame, maxDepth = treeMaxDepth, randomAttributes = randomAttributes, randomSplits = randomSplits)
        forest.append(decisionTree)
    return forest



def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    for i in prange(len(randomForest)): # // hóa với prange thay range
        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pandas.DataFrame(predictions)
    return predictions.mode(axis = 1)[0]

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Sample run

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/iris.csv")

%time
randomForest = createRandomForest(df, bootstrapSize = 10, randomAttributes = 10, randomSplits = 50, forestSize = 30, treeMaxDepth = 3)
randomForestTestResults = randomForestPredictions(df, randomForest)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [ ]:
accuracy = calculateAccuracy(randomForestTestResults, df.iloc[:, -1]) * 100
accuracy

95.33333333333334

# 3. Song song hóa sử dụng CPU & GPU **[ V3 ]**

## Decision Tree V3.0

In [ ]:
import numpy
import pandas
import random
from numba import njit, prange, cuda, float32, int32
import numba



'''

Hàm bên dưới xác định "độ tinh khiết (Pure)" của tập dữ liệu bằng cách kiểm tra xem nó

có chứa cùng một label hay không, nếu có thì return true, ngược lại return false.

'''

def checkPurity(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False

    # ...



def classifyData(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]

    # ...



def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in prange(len(uniqueValues)): # // ở đây bằng prange
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2) # tại sao dừng công thức này ?
    return potentialSplits



def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]


# CUDA kernel ref. https://nyu-cds.github.io/python-numba/05-cuda/
@cuda.jit
def calculateEntropy(data, result):

    idx = cuda.grid(2)  # Get the thread index

    if idx < len(data):
      _, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)

      probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()

      # // ở đây, thay thế hàm sum có sẵn trong python bằng vòng lặp sử dụng prange
      entropy = 0.0
      for i in prange(len(probabilities)):
          entropy -= probabilities[i] * np.log2(probabilities[i])

      result[idx] = entropy

    # ...



def calculateOverallEntropy(dataBelow, dataAbove):
    # pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    # pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    # return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)

    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))

    # Allocate device memory for entropy calculation
    entropyBelow = cuda.device_array(len(dataBelow), dtype=float)
    entropyAbove = cuda.device_array(len(dataAbove), dtype=float)

    # Launch CUDA kernels for entropy calculation
    blockSize = 128
    gridSizeBelow = (len(dataBelow) + blockSize - 1) // blockSize
    gridSizeAbove = (len(dataAbove) + blockSize - 1) // blockSize
    calculateEntropy[gridSizeBelow, blockSize](dataBelow, entropyBelow)
    calculateEntropy[gridSizeAbove, blockSize](dataAbove, entropyAbove)

    # Transfer results back to the host
    entropyBelow = entropyBelow.copy_to_host()
    entropyAbove = entropyAbove.copy_to_host()

    overallEntropy = pDataBelow * entropyBelow.sum() + pDataAbove * entropyAbove.sum()
    return overallEntropy

    # ...


def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in prange(randomSplits): # // vòng for này bằng prange
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)

            print(f'{randomSplitValue=}\n{dataBelow=}\n {dataAbove=}\n\n')
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue



def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree



def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)



def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions



def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Random Forest V2.0

In [ ]:
def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = numpy.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]



def createRandomForest(dataFrame, bootstrapSize, randomAttributes, randomSplits, forestSize = 20, treeMaxDepth = 1000):
    forest = []
    for i in prange(forestSize): # // hóa với prange thay range
        bootstrappedDataFrame = bootstrapSample(dataFrame, bootstrapSize)
        decisionTree = buildDecisionTree(bootstrappedDataFrame, maxDepth = treeMaxDepth, randomAttributes = randomAttributes, randomSplits = randomSplits)
        forest.append(decisionTree)
    return forest



def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    for i in prange(len(randomForest)): # // hóa với prange thay range
        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pandas.DataFrame(predictions)
    return predictions.mode(axis = 1)[0]

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Sample run

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/iris.csv")

# label encoder
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
df['species']= label_encoder.fit_transform(df['species'])

%time
randomForest = createRandomForest(df, bootstrapSize = 10, randomAttributes = 10, randomSplits = 50, forestSize = 30, treeMaxDepth = 3)
randomForestTestResults = randomForestPredictions(df, randomForest)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
randomSplitValue=5.65
dataBelow=array([[5. , 3.3, 1.4, 0.2, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ]])
 dataAbove=array([[7.7, 3. , 6.1, 2.3, 2. ],
       [6.1, 3. , 4.9, 1.8, 2. ],
       [7.7, 3.8, 6.7, 2.2, 2. ],
       [6.6, 2.9, 4.6, 1.3, 1. ],
       [6.3, 2.3, 4.4, 1.3, 1. ],
       [6.4, 2.8, 5.6, 2.1, 2. ],
       [6.9, 3.1, 4.9, 1.5, 1. ]])




TypingError: ignored

In [ ]:
accuracy = calculateAccuracy(randomForestTestResults, df.iloc[:, -1]) * 100
accuracy

In [ ]:
a = np.array([[1, 2, 3],
             [4,5, 6],
             [7, 8, 9]])

In [ ]:
a[:, -1]

array([3, 6, 9])